In [50]:
import numpy as np

from sentence_transformers import SentenceTransformer
from transformers import pipeline

In [27]:
model = SentenceTransformer('ggrn/e5-small-v2')

In [30]:
input1 = "Hello, my dog is cute"
output1 = model.encode(input1, normalize_embeddings=True)

In [36]:
input2 = "Hello, my cat is cute"
output2 = model.encode(input2, normalize_embeddings=True)

In [38]:
np.dot(output1, output2)

0.9242182

In [40]:
with open('textbook.txt', 'r') as f:
    textbook = f.read()

In [41]:
chunk_size = 2048

chunks = [textbook[i:i+chunk_size] for i in range(0, len(textbook), chunk_size)]

print(len(chunks))

19


In [42]:
def get_text_embedding(input):
    embeddings_batch_response = model.encode(input, normalize_embeddings=True)
    return embeddings_batch_response

text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks])

In [43]:
text_embeddings.shape

(19, 384)

In [44]:
import faiss

d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [45]:
question = "Explain Dropout"
question_embeddings = np.array([get_text_embedding(question)])

In [46]:
D, I = index.search(question_embeddings, k=2) # distance, index
retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

In [49]:
prompt = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

In [51]:
chatAgent = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [53]:
outputs = chatAgent(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

In [54]:
print(outputs[0]["generated_text"])


Context information is below.
---------------------
["d ideally like to find a sample average of all possible 2n{\\displaystyle 2^{n}} dropped-out networks; unfortunately this is unfeasible for large values of n{\\displaystyle n}. However, we can find an approximation by using the full network with each node's output weighted by a factor of p{\\displaystyle p}, so the expected value of the output of any node is the same as in the training stages. This is the biggest contribution of the dropout method: although it effectively generates 2n{\\displaystyle 2^{n}} neural nets, and as such allows for model combination, at test time only a single network needs to be tested.\nBy avoiding training all nodes on all training data, dropout decreases overfitting. The method also significantly improves training speed. This makes the model combination practical, even for deep neural networks. The technique seems to reduce node interactions, leading them to learn more robust features that better gene